Test & visualize some trained segmentation algs

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import cv2
import numpy as np
import sys, os, shutil, glob, time, random

sys.path.insert(0, '/home/nathan/caffe-segnet-crf/python')
# sys.path.insert(0, '/home/nathan/caffe-segnet-cudnn5/python')
# sys.path.insert(0, '/home/nathan/tmp/caffe-denseimagedata/python')
import caffe

sys.path.insert(0, '/home/nathan/histo-seg/v2/core')
import colorNormalization as cnorm

####################
#                  #
#                  #
caffe.set_mode_gpu()
# caffe.set_mode_cpu()
#                  #
#                  #
####################


%matplotlib inline
from matplotlib import pyplot as plt
plt.ioff()

# Run many & save RGB

In [7]:
def colorize(mask, colors):
    r = np.zeros(mask.shape, dtype=np.uint8)
    g = np.zeros(mask.shape, dtype=np.uint8)
    b = np.zeros(mask.shape, dtype=np.uint8)
    for u in np.unique(mask):
#         print 'Replacing {} with {}'.format(u, colors[u,:])
        r[mask==u] = colors[u, 0]
        g[mask==u] = colors[u, 1]
        b[mask==u] = colors[u, 2]
        
    return np.dstack([r,g,b])

# def replace_eric_labels(mask, label_levels):
#     mask_out = np.copy(mask)
#     for key in label_levels.iterkeys():
#         mask_out[mask == key] = label_levels[key]
        
#     return mask_out

expt = 'xval_set_0_10X_SegNet_CRF'
imgsource = 'xval_set_0'
dataset = 'xval_set_0_512'
size = 600

# prototxt = '/home/nathan/histo-seg/semantic-pca/code/proto/segnet_basic_deploy_merged_10X.prototxt'
# weights = '/home/nathan/histo-seg/semantic-pca/weights/xval_set_0_512/batchnorm_merged_segnet_basic_pca.caffemodel'

# prototxt = '/home/nathan/histo-seg/semantic-pca/code/segnet_basic_crfrnn_deploy_5X.prototxt'
# weights = '/home/nathan/histo-seg/semantic-pca/weights/{}/batchnorm_segnet_basic_crf_pca_20170712.SGD_iter_25000.caffemodel'.format(dataset)

prototxt = '/home/nathan/histo-seg/semantic-pca/code/segnet_crfrnn_deploy_10X.prototxt'
weights = '/home/nathan/histo-seg/semantic-pca/weights/{}/segnet_crf_pca_20170726_iter_100000.caffemodel'.format(dataset)

# prototxt = '/home/nathan/histo-seg/semantic-pca/code/segnet_deploy_10X.prototxt'
# weights = '/home/nathan/histo-seg/semantic-pca/weights/{}/batchnorm_segnet_pca_75000.caffemodel'.format(dataset)

# prototxt = '/home/nathan/histo-seg/semantic-pca/code/fcn8s_deploy_5X.prototxt'
# weights = '/home/nathan/histo-seg/semantic-pca/weights/{}/fcn_8s_PASCAL.caffemodel'.format(dataset)

net = caffe.Net(prototxt, weights, caffe.TEST)

img_list = sorted(glob.glob('/home/nathan/histo-seg/semantic-pca/data/{}/val/jpg/*.jpg'.format(imgsource)))
mask_list = sorted(glob.glob('/home/nathan/histo-seg/semantic-pca/data/{}/val/mask/*.png'.format(imgsource)))
print 'Img list: ', len(img_list)
print 'Mask list: ', len(mask_list)

colors = np.array([
    [235, 45, 35],
    [99, 220, 77],
    [55, 55, 230],
    [240, 240, 240]
])

outdir = '/home/nathan/histo-seg/semantic-pca/figures/net_outputs/{}'.format(expt)
if not os.path.exists(outdir):
    os.makedirs(outdir)
layer = 'prob'

idx = 1
timestart = time.time()
for img_name, mask_name in zip(img_list, mask_list):
#     img_name = img_list_shuffled[index]
    img_base = os.path.basename(img_name)

    img = cv2.imread(img_name)
    img = cv2.resize(img, dsize=(size, size))
    img = cnorm.normalize(img)
    img_og = np.copy(img)
    img = img.transpose((2,0,1))
    img = np.expand_dims(img, 0)

    _ = net.forward(data = img)
    seg = np.argmax(net.blobs[layer].data[0,...], 0)
    seg = colorize(seg, colors)
    
    mask = cv2.imread(mask_name, 0)
    mask = cv2.resize(mask, dsize=(size, size), interpolation=cv2.INTER_NEAREST)
    mask = colorize(mask, colors)
    
    img_og = cv2.copyMakeBorder(img_og,10,10,10,10,cv2.BORDER_CONSTANT,value=0)
    seg = cv2.copyMakeBorder(seg,10,10,10,10,cv2.BORDER_CONSTANT,value=0)
    mask = cv2.copyMakeBorder(mask,10,10,10,10,cv2.BORDER_CONSTANT,value=0)
    img_out = np.hstack([img_og, seg, mask])
    
    cv2.imwrite(os.path.join(outdir, img_base), img_out)
#     cv2.imwrite(os.path.join(outdir, img_base.replace('.jpg', '_.jpg')), seg)
    
    tdelta = time.time() - timestart
    idx += 1
    if idx % 250 == 0:
        print '{} ({:3.3f}s)'.format(idx, tdelta)

tdelta = time.time() - timestart
print 'Done in {}s'.format(tdelta)
print '{} img/s'.format(float(len(img_list)) / tdelta)

Img list:  103
Mask list:  103
Done in 33.0806260109s
3.1136049229 img/s


# Write out a bunch of masks for numbers

In [5]:
# outdir = '../figures/segnet_512_mask'

datasets = ['xval_set_0',
            'xval_set_1',
            'xval_set_2',
            'xval_set_3',
            'xval_set_4']
appendage = '_1024'
size = 300
mag = '5X'
layer = 'prob'
out_root = '/home/nathan/histo-seg/semantic-pca/analysis_segnet_basic'
prototxt = '/home/nathan/histo-seg/semantic-pca/code/segnet_basic_deploy_{}.prototxt'.format(mag)

for dataset in datasets:
    weights = '/home/nathan/histo-seg/semantic-pca/weights/{}/batchnorm_segnet_basic_pca.caffemodel'.format(dataset+appendage)

    net = caffe.Net(prototxt, weights, caffe.TEST)

    img_list = sorted(glob.glob('/home/nathan/histo-seg/semantic-pca/data/{}/val/jpg/*.jpg'.format(dataset)))
    mask_list = sorted(glob.glob('/home/nathan/histo-seg/semantic-pca/data/{}/val/mask/*.png'.format(dataset)))
    print dataset
    print 'Img list: ', len(img_list)
    print 'Mask list: ', len(mask_list)

    outdir = os.path.join(out_root, '{}'.format(dataset+appendage))
    if not os.path.exists(outdir):
        os.makedirs(outdir)

    idx = 1
    timestart = time.time()
    for img_name in img_list:
        img_base = os.path.basename(img_name).replace('jpg', 'png')

        img = cv2.imread(img_name)
        img = cv2.resize(img, dsize=(size, size))
        img = cnorm.normalize(img)
    #     img = img[:,:,::-1]
        img = img.transpose((2,0,1))
        img = np.expand_dims(img, 0)

        _ = net.forward(data = img)
    #     seg = np.argmax(net.blobs['conv1_1_D'].data[0,...], 0)
        seg = np.argmax(net.blobs[layer].data[0,...], 0)

        cv2.imwrite(os.path.join(outdir, img_base), seg)

        tdelta = time.time() - timestart
        idx += 1
        if idx % 50 == 0:
            print '{:04d} ({:3.3f}s)'.format(idx, tdelta)

    
    tdelta = time.time() - timestart
    timestr = dataset + appendage + '\n' + 'Done in {}s\n'.format(tdelta) + '{} img/s\n'.format(float(len(img_list)) / tdelta)
    print timestr
    
    with open(os.path.join(out_root, '{}_timeprofile.txt'.format(dataset+appendage)), 'w+') as f:
        f.write(timestr)
#     print dataset
#     print 'Done in {}s'.format(tdelta)
#     print '{} img/s'.format(float(len(img_list)) / tdelta)
    
    

xval_set_0
Img list:  103
Mask list:  103
0050 (5.145s)
0100 (10.296s)
xval_set_0_1024
Done in 10.7114470005s
9.61588102851 img/s

xval_set_1
Img list:  103
Mask list:  103
0050 (5.290s)
0100 (10.415s)
xval_set_1_1024
Done in 10.8649270535s
9.48004524037 img/s

xval_set_2
Img list:  102
Mask list:  102
0050 (5.312s)
0100 (9.890s)
xval_set_2_1024
Done in 10.1580691338s
10.0412783824 img/s

xval_set_3
Img list:  102
Mask list:  102
0050 (4.527s)
0100 (8.999s)
xval_set_3_1024
Done in 9.28752207756s
10.982477258 img/s

xval_set_4
Img list:  103
Mask list:  103
0050 (4.576s)
0100 (8.934s)
xval_set_4_1024
Done in 9.31234407425s
11.0605878798 img/s



# Run one

In [2]:
def add_dummy_pix(img, n = 4):
    for k in range(n):
        img[0, k] = k
    return img

# Print a color code
def plot_colorcode(n= 4):
    x = np.zeros([20,20], dtype=np.uint8)
    
    fig, ax= plt.subplots(1, 4)
    for k in range(n):
        x_ = np.copy(x)
        x_ = add_dummy_pix(x_ + k)
        print x_.mean(), x_.min(), x_.max()
        ax[k].matshow(x_)
        ax[k].set_title(k)
        ax[k].set_xticks([])
        ax[k].set_yticks([])
    
plot_colorcode()

colors = {
    0: (45, 64, 235),
    1: (64, 240, 40),
    2: (245, 56, 64),
    3: (230, 230, 230),
}

def colorize(mask, colors=colors):
    h,w = mask.shape
    x = np.zeros((h,w,3), dtype=np.uint8)
    
    for u in np.unique(mask):
        x[(mask==u), :] = colors[u]
        
    return x


def plot_bayesian_variance(segs):
    
    blob_var = np.var(segs, axis=0)
    
    n_class = blob_var.shape[0]
    fig, axs = plt.subplots(1,n_class+1, figsize=(15,3))
    im=axs[0].matshow(np.mean(blob_var, axis=0), cmap='gray_r')
    axs[0].set_xticks([]); axs[0].set_yticks([])
    axs[0].set_title('All')
    
    for k,ax in enumerate(axs[1:]):
        ax.matshow(np.squeeze(blob_var[k, :,:]), cmap='gray_r')
        ax.set_xticks([]); ax.set_yticks([])
        ax.set_title('Class {}'.format(k))
        
#     axs[-1].set_colorbar()
    fig.colorbar(im, ax=axs.ravel().tolist())
    return fig
#     plt.colorbar()


def plot_from_net(img_list, net, layer='prob', img_index=None, bayesian=False, size=None):
    
#     assert isinstance(layer, list), 'layer must be a list of layer names'
    
    ## Grab a random image
    if img_index is not None:
        index = img_index
    else:
        index = np.random.randint(len(img_list))
        
    print img_list[index]

    figs = []
    
    mask = cv2.imread(mask_list[index], 0)
    if size is not None:
        print 'Resizing'
        mask = cv2.resize(mask, dsize=(size,size), interpolation=cv2.INTER_NEAREST)
    mask = add_dummy_pix(mask)

    print img_list[index], mask_list[index]

    img = cv2.imread(img_list[index])
    if size is not None:
        print 'Resizing'
        img = cv2.resize(img, dsize=(size, size))
    img = cnorm.normalize(img)
    img_og = np.copy(img)
    img = img.transpose((2,0,1))
    img = np.expand_dims(img, 0)
    
    print 'img: ', img.shape
    print 'layer: ', layer

    net_time = time.time()
    if bayesian:
        print 'Doing the Bayesian version {} times'.format(bayesian)
        segs = []
        for _ in range(bayesian):
            _ = net.forward(data = img)
            segs.append(net.blobs[layer].data)
            
        segs = np.concatenate(segs, axis=0)
        seg = np.argmax(np.mean(segs, axis=0), axis=0)
        print 'Net finish in {}s'.format(time.time() - net_time)
        
        fig_var = plot_bayesian_variance(segs)
        figs.append(fig_var)
    else:
        print 'Doing normal net, once'
        _ = net.forward(data = img)
        seg = np.argmax(np.squeeze(net.blobs[layer].data), axis=0)
        print seg.shape
        print 'Net finish in {}s'.format(time.time() - net_time)
            
#     _ = net.forward(data = img)

#     segs = []
#     for lay in layer:
#         if bayesian:
#             seg_ = net.blobs[lay].data
#             seg_mean = np.mean(seg_, axis=0)
# #             seg_var = np.var(seg_, axis=0)
#             seg_mean = np.argmax(seg_mean, axis=0)
#             segs.append(add_dummy_pix(seg_mean))
        
#             plot_bayesian_variance(net, layer=lay)
            
#         seg = np.argmax(net.blobs[lay].data[0,...], 0)
#         segs.append(add_dummy_pix(seg))
    
    fig, axs = plt.subplots(1,3, figsize=(15,7))
    axs[0].imshow(img_og[:,:,::-1])
    axs[0].set_xticks([]); axs[0].set_yticks([])
    axs[0].set_title('H&E')
                 
        
    axs[1].matshow(add_dummy_pix(seg))
    axs[1].set_xticks([]); axs[1].set_yticks([])
    axs[1].set_title('{}'.format(layer))
#     for ax, lay, seg in zip(axs[1:-1], layer, segs):
#         ax.matshow(seg)
#         ax.set_xticks([]); axs[1].set_yticks([])
#         ax.set_title('{}'.format(lay))
    
    axs[-1].matshow(mask)
    axs[-1].set_xticks([]); axs[2].set_yticks([])
    axs[-1].set_title('Ground Truth')
    
    figs.append(fig)
    return net, index, figs






def draw_bayesian_variance(segs):
    plot_bayesian_variance(segs)
    
    blob_var = np.var(segs, axis=0)
    print 'Variance blob: ', blob_var.shape, blob_var.min(), blob_var.max()
    
    n_class = blob_var.shape[0]
    img = []
    for k in range(n_class):
        img_og = np.squeeze(blob_var[k, :,:])
        img_og *= (255.0 / img_og.max())
        img.append( cv2.copyMakeBorder(img_og,10,10,10,10,cv2.BORDER_CONSTANT,value=0) )

    img = np.concatenate(img, axis=1)
    
    img_mean = np.mean(blob_var, axis=0)
    img_mean *= (255.0 / img_mean.max())
    img_mean = cv2.copyMakeBorder(img_mean,10,10,10,10,cv2.BORDER_CONSTANT,value=0)
    img = np.concatenate([img_mean, img], axis=1)
#     img = np.log10(img)
    return 255-img


def save_from_net(img_list, net, layer='prob', img_index=None, bayesian=False, size=None):
    ## Grab a random image
    if img_index is not None:
        index = img_index
    else:
        index = np.random.randint(len(img_list))
    
    mask = cv2.imread(mask_list[index], 0)
    if size is not None:
        print 'Resizing'
        mask = cv2.resize(mask, dsize=(size,size), interpolation=cv2.INTER_NEAREST)

    print img_list[index], mask_list[index]

    img = cv2.imread(img_list[index])
    if size is not None:
        print 'Resizing'
        img = cv2.resize(img, dsize=(size, size))
    img_og = np.copy(img)
    img = cnorm.normalize(img)
    img = img.transpose((2,0,1))
    img = np.expand_dims(img, 0)
    
    print 'img: ', img.shape
    print 'layer: ', layer

    net_time = time.time()
    
    print 'Doing the Bayesian version {} times'.format(bayesian)
    segs = []
    for _ in range(bayesian):
        _ = net.forward(data = img)
        segs.append(net.blobs[layer].data)

    segs = np.concatenate(segs, axis=0)
    print 'Segs: ', segs.shape
    seg = np.argmax(np.mean(segs, axis=0), axis=0)
    print 'Argmas: ', seg.shape
    print 'Net finish in {}s'.format(time.time() - net_time)

    var_time = time.time()
    img_var = draw_bayesian_variance(segs)
    print 'Var finish in {}s'.format(time.time() - var_time)

    img_og = cv2.copyMakeBorder(img_og,10,10,10,10,cv2.BORDER_CONSTANT,value=(255,255,255))[:,:,::-1]
    mask = colorize(mask)
    mask = cv2.copyMakeBorder(mask,10,10,10,10,cv2.BORDER_CONSTANT,value=(255,255,255))
    seg = colorize(seg)
    seg = cv2.copyMakeBorder(seg,10,10,10,10,cv2.BORDER_CONSTANT,value=(255,255,255))
    
    img_eval = np.concatenate([img_og, seg, mask], 1)
    
    
    ## Force var to be the shape of eval
    var_w = img_var.shape[1]
    eva_w = img_eval.shape[1]
    ratio = float(eva_w)/var_w
    img_var = cv2.resize(img_var, dsize=(0,0), fx=ratio, fy=ratio, interpolation=cv2.INTER_LINEAR)
    img_var = np.dstack([img_var, img_var, img_var])
    img_var *= (255.0/img_var.max())
    img_var = cv2.convertScaleAbs(img_var)
    
    print 'Final var image: ', img_var.dtype, img_var.shape, img_var.min(), img_var.max()
    
    img = np.concatenate([img_var, img_eval], 0)
    return net, index, img[:,:,::-1]




def plot_filters_activations(net, layer='conv1_1', mode='f', subset=None):
    if mode == 'f':
        x = np.squeeze(net.params[layer][0].data[:,0,:,:])
        print 'Filters: shape={}'.format(x.shape)
    elif mode == 'a':
        x = np.squeeze(net.blobs[layer].data[0,...])
        
        if subset is not None and x.shape[0] > subset:
            idx = np.random.choice(range(x.shape[0]), subset)
            x = x[idx, :,:]
        print 'Activations: shape={}'.format(x.shape)
    else:
        print 'what'
        return 0

    x = (x - x.min()) / (x.max() - x.min())
    h = x.shape[1]
    s = int(np.floor(np.sqrt(x.shape[0])))
    x = x[:s**2, ...]

    #Blood magic from shelhamer
    padding = (((0, s ** 2 - x.shape[0]),
               (0, 5), (0, 5))                 # add some space between filters
               + ((0, 0),) * (x.ndim - 3))     # don't pad the last dimension (if there is one)
    x = np.pad(x, padding, mode='constant', constant_values=1)  # pad with ones (white)
    x_ = x.reshape((s, s) + x.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, x.ndim + 1)))
    x_ = x_.reshape((s * x_.shape[1], s * x_.shape[3]) + x_.shape[4:])
    
    print 'x_ plotting: {}'.format(x_.shape), x_.min(), x_.max()

    fig, ax=  plt.subplots(1,1, figsize=(13,13))
    ax.matshow(x_, cmap='jet')
    ax.set_xticks([])
    ax.set_yticks([])
    
    return fig

0.015 0 3
1.005 0 3
1.995 0 3
2.985 0 3


In [3]:
prototxt = '/home/nathan/histo-seg/semantic-pca/code/deploy_bayesian_segnet_basic_10X.prototxt'
weights = '/home/nathan/histo-seg/semantic-pca/weights/whole_set_512/batchnorm_bayesian_segnet_pca_50000.caffemodel'

# prototxt = '/home/nathan/histo-seg/semantic-pca/code/segnet_basic_deploy.prototxt'
# weights = '/home/nathan/histo-seg/semantic-pca/weights/whole_set_512/batchnorm_segnet_pca_200000.caffemodel'

imgsource = 'xval_set_4'
img_list = sorted(glob.glob('/home/nathan/histo-seg/semantic-pca/data/{}/val/jpg/*.jpg'.format(imgsource)))
mask_list = sorted(glob.glob('/home/nathan/histo-seg/semantic-pca/data/{}/val/mask/*.png'.format(imgsource)))
print 'Img list: ', len(img_list)
print 'Mask list: ', len(mask_list)
net = caffe.Net(prototxt, weights, caffe.TEST)

print '\nNet:'
for k in net.blobs.iterkeys():
    print k, net.blobs[k].data.shape

Img list:  103
Mask list:  103

Net:
data (1, 3, 600, 600)
norm (1, 3, 600, 600)
conv1 (1, 64, 600, 600)
pool1 (1, 64, 300, 300)
pool1_mask (1, 64, 300, 300)
conv2 (1, 64, 300, 300)
pool2 (1, 64, 150, 150)
pool2_mask (1, 64, 150, 150)
conv3 (1, 64, 150, 150)
pool3 (1, 64, 75, 75)
pool3_mask (1, 64, 75, 75)
conv4 (1, 64, 75, 75)
pool4 (1, 64, 38, 38)
pool4_mask (1, 64, 38, 38)
upsample4 (1, 64, 75, 75)
conv_decode4 (1, 64, 75, 75)
upsample3 (1, 64, 150, 150)
conv_decode3 (1, 64, 150, 150)
upsample2 (1, 64, 300, 300)
conv_decode2 (1, 64, 300, 300)
upsample1 (1, 64, 600, 600)
conv_decode1 (1, 64, 600, 600)
dense_softmax_inner_prod (1, 4, 600, 600)
prob (1, 4, 600, 600)


In [4]:
for index in range(len(img_list)):
# for index in range(10):
    print '\n'*3
    net, index, img = save_from_net(img_list, net, layer='prob', img_index=index, bayesian=50, size=600)
    cv2.imwrite('/home/nathan/histo-seg/semantic-pca/figures/bayesian_segnet_50k/4_{}.png'.format(index), img)

# plt.imshow(img)

# plt.figure()
# plt.imshow(img_pred)
# plt.show()
# plt.show()
#     figs[0].savefig('/home/nathan/histo-seg/semantic-pca/figures/bayesian_segnet_100k/3_{}_var.png'.format(index), bbox_inches='tight')
#     figs[1].savefig('/home/nathan/histo-seg/semantic-pca/figures/bayesian_segnet_100k/3_{}_pred.png'.format(index), bbox_inches='tight')






Resizing
/home/nathan/histo-seg/semantic-pca/data/xval_set_4/val/jpg/0005.jpg /home/nathan/histo-seg/semantic-pca/data/xval_set_4/val/mask/0005.png
Resizing
img:  (1, 3, 600, 600)
layer:  prob
Doing the Bayesian version 50 times
Segs:  (50, 4, 600, 600)
Argmas:  (600, 600)
Net finish in 3.70003986359s
Variance blob:  (4, 600, 600) 0.0 5.11591e-13
Var finish in 1.41230082512s
Final var image:  uint8 (372, 1860, 3) 0 255




Resizing
/home/nathan/histo-seg/semantic-pca/data/xval_set_4/val/jpg/0007.jpg /home/nathan/histo-seg/semantic-pca/data/xval_set_4/val/mask/0007.png
Resizing
img:  (1, 3, 600, 600)
layer:  prob
Doing the Bayesian version 50 times
Segs:  (50, 4, 600, 600)
Argmas:  (600, 600)
Net finish in 3.57124614716s
Variance blob:  (4, 600, 600) 0.0 5.11591e-13
Var finish in 1.48907804489s
Final var image:  uint8 (372, 1860, 3) 0 255




Resizing
/home/nathan/histo-seg/semantic-pca/data/xval_set_4/val/jpg/0012.jpg /home/nathan/histo-seg/semantic-pca/data/xval_set_4/val/mask/001

/usr/lib/python2.7/dist-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Variance blob:  (4, 600, 600) 0.0 5.11591e-13
Var finish in 1.52778601646s
Final var image:  uint8 (372, 1860, 3) 0 255




Resizing
/home/nathan/histo-seg/semantic-pca/data/xval_set_4/val/jpg/0098.jpg /home/nathan/histo-seg/semantic-pca/data/xval_set_4/val/mask/0098.png
Resizing
img:  (1, 3, 600, 600)
layer:  prob
Doing the Bayesian version 50 times
Segs:  (50, 4, 600, 600)
Argmas:  (600, 600)
Net finish in 3.67684698105s
Variance blob:  (4, 600, 600) 0.0 5.11591e-13
Var finish in 1.28396296501s
Final var image:  uint8 (372, 1860, 3) 0 255




Resizing
/home/nathan/histo-seg/semantic-pca/data/xval_set_4/val/jpg/0100.jpg /home/nathan/histo-seg/semantic-pca/data/xval_set_4/val/mask/0100.png
Resizing
img:  (1, 3, 600, 600)
layer:  prob
Doing the Bayesian version 50 times
Segs:  (50, 4, 600, 600)
Argmas:  (600, 600)
Net finish in 3.63704800606s
Variance blob:  (4, 600, 600) 0.0 5.11591e-13
Var finish in 1.28185606003s
Final var image:  uint8 (372, 1860, 3) 0 255




Resizing
/home/nathan/